In [1]:
#| echo: false
# Libraries
import pandas as pd
import numpy as np
from redcap import Project
from tableone import TableOne

In [2]:
# REDCap setup
wd = '/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Laurel Messer/SEEDS/'
api = pd.read_table(wd+'Data_Raw/api_token.txt',header=None)
api_url = 'https://redcap.ucdenver.edu/api/'
api_key = api.iloc[0,0]
project = Project(api_url, api_key)
# Data cleaning
df = pd.DataFrame(project.export_records(raw_or_label='label',raw_or_label_headers='label'))
# Blanks to missing
df[df==''] = np.NaN
# Real records only
df = df[df['real_record'] == 'Real']
# Change specialty types per Laurel
df.loc[df['otherspecialty'] == 'Pediatric Diabetology','specialty'] = 'Pediatric Endocrinology'
df.loc[df['otherspecialty'] == 'Adult Gerontology Primary Care','specialty'] = 'Primary care-family medicine'
# Combine levels 
df['specialty'].replace(dict.fromkeys(['Primary care-med/peds', 'Primary care-family medicine',
                                       'Primary care-pediatrics','Primary care-internal medicine'],['Primary Care']),inplace=True)
df['providertype'].replace(['Physician (MD, DO or equivalent)','Advance practice professional (Physician assistant, nurse practitioner)',
                           'Diabetes care and education specialist (diabetes educator, nurse, dietitian, other)',
                           'Endocrinology fellow','Medical resident'],
                           ['Prescriber','Prescriber','Non-Prescriber','Prescriber','Non-Prescriber'],inplace=True)
# Combine levels of practice type
df['practicetype'].replace(dict.fromkeys(['Health maintenance organization (e.g. Kaiser Permanente)',
                                          'Federally Qualified Health Canter','Veterans administration'],
                           ['Other']),inplace=True)
# Rename columns
df.rename(columns={"staffsupport___0":"Clinical pharmacist","staffsupport___1":"DE","staffsupport___2":"Care manager","staffsupport___3":"APP","staffsupport___4":"Behavioral health",
                  "patientage___0":"Pediatric patients with diabetes","patientage___1":"Adult patients with diabetes",
                  "diabetestype___0":"Type 1 diabetes","diabetestype___2":"Type 2 diabetes","diabetestype___3":"Other diabetes (gestational, MODY, etc.)"},inplace=True)
# Drop 'Other' providers
df = df[(df['specialty'] != 'Other') & (pd.notna(df['specialty']))]

# Table 1: Participant Characteristics

In [3]:
#| warning: false
# Categorical variables
cat_vars = ['gender','race','practicetype','specialty','practicesetting',
             'medicare','medicaid','privateins','uninsured',
           "Clinical pharmacist","DE","Care manager",
           "APP","Behavioral health"]
# Continuous variables
con_vars = ['age','practiceyears','cgmpatientratio','pumppatientratio',
             'penpatientratio','hclpatientratio','educationtime']
# Convert to numeric
df[con_vars] = df[con_vars].apply(pd.to_numeric)
# Table
TableOne(data=df,columns=con_vars+cat_vars,categorical=cat_vars,groupby='specialty')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(l

Grouped by specialty                                                                      
                                                                                  Missing      Overall Adult endocrinology Pediatric Endocrinology Primary Care
n                                                                                                  240                  89                      36          115
age, mean (SD)                                                                          0  44.8 (11.1)          41.8 (9.6)             42.8 (11.7)  47.7 (11.3)
practiceyears, mean (SD)                                                                0  13.8 (10.3)          10.8 (9.2)             13.1 (12.1)  16.3 (10.0)
cgmpatientratio, mean (SD)                                                              6  42.4 (28.8)         53.0 (21.1)             72.3 (15.4)  25.4 (25.6)
pumppatientratio, mean (SD)                                                             6  20.3 (18.6)         22.1 (14.3)             42.7 (22.9)  12.2 (13.5)
penpatientratio, mean (SD)                                                              6   9.9 (16.4)          6.9 (10.8)             10.9 (11.3)  11.9 (20.4)
hclpatientratio, mean (SD)                                                              6  22.3 (23.1)         29.2 (21.2)             45.4 (18.4)  10.1 (17.7)
educationtime, mean (SD)                                                                6  20.3 (16.3)         21.6 (17.6)             27.9 (14.2)  17.0 (15.1)
gender, n (%)               Female                                                      0   150 (62.5)           65 (73.0)               29 (80.6)    56 (48.7)
                            Male                                                             90 (37.5)           24 (27.0)                7 (19.4)    59 (51.3)
race, n (%)                 Asian/Pacific Islander                                      0    40 (16.7)           16 (18.0)                4 (11.1)    20 (17.4)
                            Other                                                             19 (7.9)           12 (13.5)                 2 (5.6)      5 (4.3)
                            White                                                           175 (72.9)           61 (68.5)               29 (80.6)    85 (73.9)
                            Native American                                                    1 (0.4)                                     1 (2.8)             
                            Black                                                              5 (2.1)                                                  5 (4.3)
practicetype, n (%)         Academic Medical Center/faculty practice                    0    86 (35.8)           36 (40.4)               25 (69.4)    25 (21.7)
                            Clinician-owned solo or group practice                           58 (24.2)           13 (14.6)                 2 (5.6)    43 (37.4)
                            Hospital or Health system owned                                  71 (29.6)           31 (34.8)                7 (19.4)    33 (28.7)
                            Other                                                            25 (10.4)            9 (10.1)                 2 (5.6)    14 (12.2)
specialty, n (%)            Adult endocrinology                                         0    89 (37.1)          89 (100.0)                                     
                            Pediatric Endocrinology                                          36 (15.0)                                  36 (100.0)             
                            Primary Care                                                    115 (47.9)                                              115 (100.0)
practicesetting, n (%)      Rural                                                       0    36 (15.0)             4 (4.5)                 1 (2.8)    31 (27.0)
                            Suburban                                 

# Table 2: Description of diabetes and insulin requiring patients

In [4]:
#| warning: false
# Continuous variables
con_vars = ["patientsoninsulin","cgmpatientratio","penpatientratio","pumppatientratio","hclpatientratio"]
# Categorical variables
cat_vars = ["Pediatric patients with diabetes","Adult patients with diabetes",
            "Type 1 diabetes","Type 2 diabetes","Other diabetes (gestational, MODY, etc.)",
           "mdi_type_sliding","mdi_type_fixed_sliding","mdi_type_est_sliding","mdi_type_ratios"]
# Convert to numeric
df[con_vars] = df[con_vars].apply(pd.to_numeric)
TableOne(data=df,columns=con_vars+cat_vars,categorical=cat_vars,groupby='specialty')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(l

Grouped by specialty                                                                      
                                                                       Missing      Overall Adult endocrinology Pediatric Endocrinology Primary Care
n                                                                                       240                  89                      36          115
patientsoninsulin, mean (SD)                                                 2  65.6 (64.1)        103.2 (73.9)             63.6 (61.3)  37.9 (37.7)
cgmpatientratio, mean (SD)                                                   6  42.4 (28.8)         53.0 (21.1)             72.3 (15.4)  25.4 (25.6)
penpatientratio, mean (SD)                                                   6   9.9 (16.4)          6.9 (10.8)             10.9 (11.3)  11.9 (20.4)
pumppatientratio, mean (SD)                                                  6  20.3 (18.6)         22.1 (14.3)             42.7 (22.9)  12.2 (13.5)
hclpatientratio, mean (SD)                                                   6  22.3 (23.1)         29.2 (21.2)             45.4 (18.4)  10.1 (17.7)
Pediatric patients with diabetes, n (%)         Checked                      0    76 (31.7)             4 (4.5)              36 (100.0)    36 (31.3)
                                                Unchecked                        164 (68.3)           85 (95.5)                            79 (68.7)
Adult patients with diabetes, n (%)             Checked                      0   219 (91.2)           86 (96.6)               20 (55.6)   113 (98.3)
                                                Unchecked                          21 (8.8)             3 (3.4)               16 (44.4)      2 (1.7)
Type 1 diabetes, n (%)                          Checked                      0   216 (90.0)           86 (96.6)              36 (100.0)    94 (81.7)
                                                Unchecked                         24 (10.0)             3 (3.4)                            21 (18.3)
Type 2 diabetes, n (%)                          Checked                      0   232 (96.7)           87 (97.8)               32 (88.9)   113 (98.3)
                                                Unchecked                           8 (3.3)             2 (2.2)                4 (11.1)      2 (1.7)
Other diabetes (gestational, MODY, etc.), n (%) Checked                      0   124 (51.7)           66 (74.2)               23 (63.9)    35 (30.4)
                                                Unchecked                        116 (48.3)           23 (25.8)               13 (36.1)    80 (69.6)
mdi_type_sliding, n (%)                         0%                           6    41 (17.5)           19 (22.1)                 3 (8.8)    19 (16.7)
                                                1-25%                            147 (62.8)           53 (61.6)               26 (76.5)    68 (59.6)
                                                26-50%                            29 (12.4)            9 (10.5)                 1 (2.9)    19 (16.7)
                                                51-75%                             12 (5.1)             3 (3.5)                 1 (2.9)      8 (7.0)
                                                76-100%                             5 (2.1)             2 (2.3)                 3 (8.8)             
mdi_type_fixed_sliding, n (%)                   1-25%                        6    95 (40.6)           28 (32.6)               24 (70.6)    43 (37.7)
                                                26-50%                            68 (29.1)           33 (38.4)                 2 (5.9)    33 (28.9)
                                                51-75%                            45 (19.2)           22 (25.6)                            23 (20.2)
                                                76-100%                             9 (3.8)             3 (3.5)                              6 (5.3)
                                   